In [ ]:
# syft absolute
import syft as sy

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=False,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

In [ ]:
high_client.worker_pools

In [ ]:
requested_keys = ["external_registry", "whatever", "service_account_bigquery_mock"]
secrets = sy.get_nb_secrets(requested_keys = requested_keys)

In [ ]:
external_registry = "docker.io"

In [ ]:
worker_docker_tag = f"openmined/bigquery:{sy.__version__}"
worker_docker_tag

In [ ]:
docker_config = sy.PrebuiltWorkerConfig(tag=f"{external_registry}/{worker_docker_tag}")
docker_config

In [ ]:
submit_result = high_client.api.services.worker_image.submit(
    worker_config=docker_config
)
submit_result

In [ ]:
worker_image = high_client.images.get_all()[1]
worker_image

In [ ]:
registry_add_result = high_client.api.services.image_registry.add(external_registry)
registry_add_result

In [ ]:
registries = high_client.api.services.image_registry.get_all()
registries

In [ ]:
registry_uid = next((r.id for r in registries if r.url == external_registry), None)

In [ ]:
worker_pool_name = "bigquery-pool"

In [ ]:
high_client.api.services.worker_pool.launch(
    pool_name=worker_pool_name,
    image_uid=worker_image.id,
    num_workers=1,
)

In [ ]:
high_client.worker_pools.scale(number=5, pool_name=worker_pool_name)

high_client.worker_pools

In [ ]:
base_worker_image = high_client.images.get_all()[0]
base_worker_image

In [ ]:
high_client.register(
    email="data_scientist@openmined.org",
    password="verysecurepassword",
    password_verify="verysecurepassword",
    name="John Doe",
)

In [ ]:
high_client.settings.allow_guest_signup(enable=False)

In [ ]:
worker_dockerfile = f"""
FROM {str(base_worker_image.image_identifier)}

RUN uv pip install db-dtypes google-cloud-bigquery

""".strip()
worker_dockerfile

In [ ]:
docker_tag = str(base_worker_image.image_identifier).replace(
    "backend", "worker-bigquery"
)
docker_tag

In [ ]:
# # can't shutdown because in python the workers are gone in the next notebook
# assert False

In [ ]:
# server.land()

In [ ]:
# !echo "$worker_dockerfile" | docker build -t $docker_tag -q -

In [ ]:
# !docker image ls | grep bigquery